In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Collection

In [ ]:
import zipfile

# Specify the path to the zip file
zip_file_path = '/content/drive/MyDrive/Data_defect.zip' #/kaggle/input/data-shoes/Split_Data-20250130T053624Z-001'

extract_dir = '/content/'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("File extracted successfully!\\")

File extracted successfully!\


In [ ]:
import zipfile

# Specify the path to the zip file
zip_file_path = '/content/drive/MyDrive/New_shoe/Data-defect.zip' #/kaggle/input/data-shoes/Split_Data-20250130T053624Z-001'

extract_dir = '/content/drive/MyDrive/New_shoe/Data-defect'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("File extracted successfully!\\")

File extracted successfully!\


In [ ]:
zip_file_path = '/kaggle/input/data-shoes/Split_Data-20250130T053624Z-001'
if os.path.isfile(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
else:
    print(f"{zip_file_path} is a directory, not a zip file.")


/kaggle/input/data-shoes/Split_Data-20250130T053624Z-001 is a directory, not a zip file.


In [ ]:
import os

# Check the contents of the directory
zip_dir = '/kaggle/input/data-shoes/'
print(os.listdir(zip_dir))


['data-kaggle.yaml', 'Split_Data-20250130T053624Z-001']


In [ ]:
import zipfile

# Specify the path to the zip file
zip_file_path = '/content/Dataset.zip'

extract_dir = '/content/'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("File extracted successfully!\\")

File extracted successfully!\


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Data Pre-Processing

In [ ]:
original_images_path= dataset_dir

In [ ]:
import cv2

In [ ]:
categories = ['Normal', 'Defect']  # List of categories

In [ ]:
# Path to save the enhanced images
enhanced_images_path = '/content/drive/MyDrive/Data_Shoes/Yolo_data/Enhanced'

# Create the directory if it doesn't exist
if not os.path.exists(enhanced_images_path):
    os.makedirs(enhanced_images_path)

## Image Augmentation

In [ ]:
!pip install albumentations

In [ ]:
import os
import cv2
import albumentations as A

# Paths
input_images_path = "/content/drive/MyDrive/Data_Shoes/Yolo_data/Data/images (1)"
input_labels_path = "/content/drive/MyDrive/Data_Shoes/Yolo_data/Data/labels (1)"
output_images_path = "/content/drive/MyDrive/Data_Shoes/Yolo_data/Augmented/images"
output_labels_path = "/content/drive/MyDrive/Data_Shoes/Yolo_data/Augmented/labels"

# Create output directories if they don't exist
os.makedirs(output_images_path, exist_ok=True)
os.makedirs(output_labels_path, exist_ok=True)

# Augmentation pipeline
transform = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.3),
        A.Rotate(limit=15, p=0.4),
        A.RandomBrightnessContrast(p=0.4),
    ],
    bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]),
)

# Target number of images for each category
target_images_per_category = {
    0: 800,  # Normal
    1:800,  # Defect
}

# Categories dictionary
categories = {0: "Normal", 1: "Defect"}

# List all image files in the images folder
image_files = [f for f in os.listdir(input_images_path) if f.endswith(".jpg")]

# Function to get the category of an image from its label file
def get_category_from_label(label_path):
    with open(label_path, "r") as f:
        lines = f.readlines()
        if not lines:  # Check if the label file is empty
            return 0  # Assume 'Normal' category if no labels are present
        class_id = int(lines[0].split()[0])  # First entry corresponds to the class
        return class_id if class_id in categories else 0  # Default to 'Normal' if invalid class_id

# Organize images by category
images_by_category = {0: [], 1: []}
for img_file in image_files:
    img_path = os.path.join(input_images_path, img_file)
    label_path = os.path.join(input_labels_path, img_file.replace(".jpg", ".txt"))

    # Get the category for the image
    if os.path.exists(label_path):
        category = get_category_from_label(label_path)
    else:
        category = 0

    # Add the image to the appropriate category list
    images_by_category[category].append(img_file)

# Process each category and augment images to meet the target
for category, image_list in images_by_category.items():
    category_name = categories[category]
    current_image_count = len(image_list)
    target_count = target_images_per_category[category]
    augmentations_needed = target_count - current_image_count

    print(f"Processing category: {category_name} | Current: {current_image_count}, Target: {target_count}, Needed: {augmentations_needed}")

    if augmentations_needed > 0:
        for img_file in image_list:
            img_path = os.path.join(input_images_path, img_file)
            image = cv2.imread(img_path)

            label_path = os.path.join(input_labels_path, img_file.replace(".jpg", ".txt"))
            bboxes = []
            class_labels = []

            # Read bounding boxes if label file exists and is not empty
            if os.path.exists(label_path):
                with open(label_path, "r") as f:
                    for line in f.readlines():
                        parts = line.strip().split()
                        class_id = int(parts[0])
                        x_center, y_center, width, height = map(float, parts[1:])
                        bboxes.append([x_center, y_center, width, height])
                        class_labels.append(class_id)

            # Generate the required number of augmentations
            for i in range(augmentations_needed):
                augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
                augmented_image = augmented["image"]
                augmented_bboxes = augmented["bboxes"]

                # Save augmented image
                aug_img_name = f"aug_{category}_{i}_{img_file}"
                cv2.imwrite(os.path.join(output_images_path, aug_img_name), augmented_image)

                # Save augmented labels (only if there are bounding boxes)
                if augmented_bboxes:
                    aug_label_name = f"aug_{category}_{i}_{img_file.replace('.jpg', '.txt')}"
                    with open(os.path.join(output_labels_path, aug_label_name), "w") as f:
                        for bbox, class_id in zip(augmented_bboxes, class_labels):
                            f.write(f"{class_id} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")

                augmentations_needed -= 1
                if augmentations_needed == 0:
                    break
    else:
        print(f"Category '{category_name}' already meets or exceeds the target count. No augmentation needed.")

print("Augmentation completed!")

In [ ]:
import os
import shutil

# Paths
original_images_path = "/content/drive/MyDrive/Data_Shoes/Yolo_data/Data/images"
original_labels_path = "/content/drive/MyDrive/Data_Shoes/Yolo_data/Data/labels"
augmented_images_path = "/content/drive/MyDrive/Data_Shoes/Yolo_data/Augmented/images"
augmented_labels_path = "/content/drive/MyDrive/Data_Shoes/Yolo_data/Augmented/labels"
combined_images_path = "/content/drive/MyDrive/Data_Shoes/Yolo_data/Combined/images"
combined_labels_path = "/content/drive/MyDrive/Data_Shoes/Yolo_data/Combined/labels"

# Create combined output directories if they don't exist
os.makedirs(combined_images_path, exist_ok=True)
os.makedirs(combined_labels_path, exist_ok=True)

# Function to copy files from one directory to another
def copy_files(src_dir, dest_dir):
    for file_name in os.listdir(src_dir):
        src_file = os.path.join(src_dir, file_name)
        dest_file = os.path.join(dest_dir, file_name)

        if os.path.isfile(src_file):
            shutil.copy(src_file, dest_file)

# Combine original and augmented images
print("Combining original and augmented images...")
copy_files(original_images_path, combined_images_path)
copy_files(augmented_images_path, combined_images_path)

# Combine original and augmented labels
print("Combining original and augmented labels...")
copy_files(original_labels_path, combined_labels_path)
copy_files(augmented_labels_path, combined_labels_path)

print("Combining completed!")

In [ ]:
# Function to count the number of files in a directory
def count_files(directory, extension):
    return len([f for f in os.listdir(directory) if f.endswith(extension)])

# Count images and labels in the combined folders
image_count = count_files(combined_images_path, ".jpg")  # Assuming image files are .jpg
label_count = count_files(combined_labels_path, ".txt")  # Assuming label files are .txt

print(f"Number of images in combined folder: {image_count}")
print(f"Number of labels in combined folder: {label_count}")

In [ ]:
import os

# Define image and label folders
image_folder = "/content/drive/MyDrive/Data_Shoes/Yolo_data/Combined/images"
label_folder = "/content/drive/MyDrive/Data_Shoes/Yolo_data/Combined/labels"

# Get the list of image and label files
images = os.listdir(image_folder)
labels = os.listdir(label_folder)

# Extract image base names (without extensions) for comparison
image_names = [os.path.splitext(image)[0] for image in images]
label_names = [os.path.splitext(label)[0] for label in labels]

# Find images without labels
missing_labels = [image for image in image_names if image not in label_names]

# For each image without a label, create a corresponding label file
for image in missing_labels:
    label_file = image + '.txt'  # The label file will have the same base name as the image
    label_path = os.path.join(label_folder, label_file)

    # Create a label file indicating "No Defect" or leave it blank
    with open(label_path, 'w') as file:
        file.write("Normal")  # You can leave it blank if needed, or add specific information
    print(f"Created label for {image}: Normal")

print(f"Number of missing labels created: {len(missing_labels)}")


## Combined Data

In [ ]:
import os

# Define image and label folders
image_folder = "/content/drive/MyDrive/Data_Shoes/Yolo_data/Combined/images"
label_folder = "/content/drive/MyDrive/Data_Shoes/Yolo_data/Combined/labels"

# Get the list of image and label files
images = os.listdir(image_folder)
labels = os.listdir(label_folder)

# Extract image base names (without extensions) for comparison
image_names = [os.path.splitext(image)[0] for image in images]
label_names = [os.path.splitext(label)[0] for label in labels]

# Find extra labels (labels that don't correspond to any image)
extra_labels = [label for label in label_names if label not in image_names]

# If there is an extra label, remove it
for extra_label in extra_labels:
    label_path = os.path.join(label_folder, extra_label + '.txt')
    if os.path.exists(label_path):
        os.remove(label_path)
        print(f"Removed extra label: {extra_label}.txt")

print(f"Number of extra labels removed: {len(extra_labels)}")


In [ ]:
# Count images and labels in the combined folders
image_count = count_files(combined_images_path, ".jpg")  # Assuming image files are .jpg
label_count = count_files(combined_labels_path, ".txt")  # Assuming label files are .txt

print(f"Number of images in combined folder: {image_count}")
print(f"Number of labels in combined folder: {label_count}")

## Image Enhancement

In [ ]:
import cv2
# Function to adjust contrast and brightness of the image
def adjust_contrast(image, alpha= 3,beta=  0.2):
    """
    Increase contrast by adjusting alpha (contrast) and beta (brightness).
    :param alpha: Contrast factor (greater than 1 to increase contrast).
    :param beta: Brightness factor (can be adjusted for better lighting).
    :return: Image with adjusted contrast.
    """
    return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

In [ ]:
# Load the original and enhanced images
import matplotlib.pyplot as plt

img = cv2.imread('/content/drive/MyDrive/Data_Shoes/Yolo_data/Combined/images/20211110_185302.jpg') # Removed the extra space
enhanced_img = adjust_contrast(img, 3, 0.2)
# Convert BGR to RGB for correct display in matplotlib
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
enhanced_img_rgb = cv2.cvtColor(enhanced_img, cv2.COLOR_BGR2RGB)

# Plot both original and enhanced images
plt.figure(figsize=(12, 6))

# Original image plot
plt.subplot(1, 2, 1)
plt.imshow(img_rgb)
plt.title("Original Image")
plt.axis('off')

# Enhanced image plot
plt.subplot(1, 2, 2)
plt.imshow(enhanced_img_rgb)
plt.title("Enhanced Image")
plt.axis('off')

plt.show()

In [ ]:
3# Load the original and enhanced images
import matplotlib.pyplot as plt

img = cv2.imread('/content/drive/MyDrive/Data_Shoes/Yolo_data/Combined/images/aug_0_125_20211110_185254.jpg') # Removed the extra space
enhanced_img = adjust_contrast(img, 3, 0.2)
# Convert BGR to RGB for correct display in matplotlib
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
enhanced_img_rgb = cv2.cvtColor(enhanced_img, cv2.COLOR_BGR2RGB)

# Plot both original and enhanced images
plt.figure(figsize=(12, 6))

# Original image plot
plt.subplot(1, 2, 1)
plt.imshow(img_rgb)
plt.title("Original Image")
plt.axis('off')

# Enhanced image plot
plt.subplot(1, 2, 2)
plt.imshow(enhanced_img_rgb)
plt.title("Enhanced Image")
plt.axis('off')

plt.show()

In [ ]:
img = cv2.imread('/content/drive/MyDrive/Data_Shoes/Yolo_data/Combined/images/aug_0_143_20211110_185254.jpg') # Removed the extra space
enhanced_img = adjust_contrast(img,3 , 0.2)
# Convert BGR to RGB for correct display in matplotlib
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
enhanced_img_rgb = cv2.cvtColor(enhanced_img, cv2.COLOR_BGR2RGB)

# Plot both original and enhanced images
plt.figure(figsize=(12, 6))

# Original image plot
plt.subplot(1, 2, 1)
plt.imshow(img_rgb)
plt.title("Original Image")
plt.axis('off')

# Enhanced image plot
plt.subplot(1, 2, 2)
plt.imshow(enhanced_img_rgb)
plt.title("Enhanced Image")
plt.axis('off')

plt.show()

In [ ]:
import cv2
import os
from shutil import copyfile

# Paths for the Combined images and labels folder
combined_images_path = '/content/drive/MyDrive/Data_Shoes/Yolo_data/Combined/images'
combined_labels_path = '/content/drive/MyDrive/Data_Shoes/Yolo_data/Combined/labels'

# Path to save the enhanced images
enhanced_images_path = '/content/drive/MyDrive/Data_Shoes/Yolo_data/Enhanced_data/images'
enhanced_labels_path = '/content/drive/MyDrive/Data_Shoes/Yolo_data/Enhanced_data/labels'

# Create the directory for enhanced images and labels if they don't exist
os.makedirs(enhanced_images_path, exist_ok=True)
os.makedirs(enhanced_labels_path, exist_ok=True)

# Process the images from the Combined folder
def enhance_images_from_combined():
    image_files = [f for f in os.listdir(combined_images_path) if f.endswith('.jpg')]

    for img_file in image_files:
        # Image and label paths
        img_path = os.path.join(combined_images_path, img_file)
        label_path = os.path.join(combined_labels_path, img_file.replace('.jpg', '.txt'))

        # Read the image
        image = cv2.imread(img_path)

        # Adjust contrast and brightness
        enhanced_image = adjust_contrast(image)

        # Save the enhanced image to the 'Enhanced' folder
        enhanced_img_name = f"enhanced_{img_file}"
        enhanced_image_path = os.path.join(enhanced_images_path, enhanced_img_name)
        cv2.imwrite(enhanced_image_path, enhanced_image)

        # Copy the label file (since the label remains the same)
        if os.path.exists(label_path):
            enhanced_label_name = f"enhanced_{img_file.replace('.jpg', '.txt')}"
            enhanced_label_path = os.path.join(enhanced_labels_path, enhanced_label_name)
            copyfile(label_path, enhanced_label_path)

# Run the enhancement function
enhance_images_from_combined()

# Verify the count of enhanced images and labels
enhanced_image_count = len([f for f in os.listdir(enhanced_images_path) if f.endswith('.jpg')])
enhanced_label_count = len([f for f in os.listdir(enhanced_labels_path) if f.endswith('.txt')])

print(f"Enhanced images count: {enhanced_image_count}")
print(f"Enhanced labels count: {enhanced_label_count}")

In [ ]:
# Paths for enhanced images and labels
import random
enhanced_images_path = '/content/drive/MyDrive/Data_Shoes/Yolo_data/Enhanced_data/images'
enhanced_labels_path = '/content/drive/MyDrive/Data_Shoes/Yolo_data/Enhanced_data/labels'

# Paths for train and validation split
train_images_path = '/content/drive/MyDrive/Data_Shoes/Yolo_data/train/images'
train_labels_path = '/content/drive/MyDrive/Data_Shoes/Yolo_data/train/labels'
val_images_path = '/content/drive/MyDrive/Data_Shoes/Yolo_data/val/images'
val_labels_path = '/content/drive/MyDrive/Data_Shoes/Yolo_data/val/labels'

# Create directories for train and validation sets
os.makedirs(train_images_path, exist_ok=True)
os.makedirs(train_labels_path, exist_ok=True)
os.makedirs(val_images_path, exist_ok=True)
os.makedirs(val_labels_path, exist_ok=True)

# Function to split data into train and test (80-20 split)
def train_test_split(image_dir, label_dir, train_img_dir, train_lbl_dir, val_img_dir, val_lbl_dir, split_ratio=0.8):
    image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]

    # Shuffle the images to randomize
    random.shuffle(image_files)

    # Calculate the split index
    split_index = int(len(image_files) * split_ratio)

    # Split the images into training and validation sets
    train_images = image_files[:split_index]
    val_images = image_files[split_index:]

    # Copy the images and labels into respective directories
    for img_file in train_images:
        shutil.copy(os.path.join(image_dir, img_file), train_img_dir)
        shutil.copy(os.path.join(label_dir, img_file.replace('.jpg', '.txt')), train_lbl_dir)

    for img_file in val_images:
        shutil.copy(os.path.join(image_dir, img_file), val_img_dir)
        shutil.copy(os.path.join(label_dir, img_file.replace('.jpg', '.txt')), val_lbl_dir)

    print(f"Train set size: {len(train_images)} images")
    print(f"Validation set size: {len(val_images)} images")

# Perform the train-test split
train_test_split(enhanced_images_path, enhanced_labels_path, train_images_path, train_labels_path, val_images_path, val_labels_path)

# Now, create the 'data.yaml' file for YOLOv5

# Paths for the YAML file
yaml_file_path = '/content/drive/MyDrive/Data_Shoes/Yolo_data/data.yaml'

# Number of classes (based on your dataset)
num_classes = 2

# Define the YAML file content
yaml_content = f"""
train: '/content/drive/MyDrive/Data_Shoes/Yolo_data/train/images'
val: '/content/drive/MyDrive/Data_Shoes/Yolo_data/val/images'
nc: 2
names: ['Normal', 'Defect']
"""

# Write the YAML file
with open(yaml_file_path, 'w') as f:
    f.write(yaml_content)

print(f"YOLOv5 data.yaml file created at: {yaml_file_path}")

#Yolov5s

In [ ]:
import yaml

# Define the data for the YAML file
data = {
    'train': '/content/Dataset/train/images',
    'val': '/content/Dataset/val/images',
    'test': '/content/Dataset/test/images',
    'nc': 1,
    'names': ['Defect']
}

# Specify the path where you want to save the YAML file
yaml_file_path = '/content/data.yaml'

# Write the data to the YAML file
with open(yaml_file_path, 'w') as yaml_file:
    yaml.dump(data, yaml_file)

print(f"YAML file created at: {yaml_file_path}")


YAML file created at: /content/data.yaml


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
cat /content/data.yaml


train: '/content/Dataset/train/images'
val: '/content/Dataset/val/images'
test: '/content/Dataset/test/images'
nc: 1
names: ['Defect']


In [ ]:
from ultralytics import YOLO
import torch

# Path to your dataset's YAML file
data_yaml_path = '/content/data.yaml'  # Ensure this YAML file is correctly configured
# Load YOLOv8 model with pre-trained weights
model = YOLO("yolov5s.pt")

# Train the model
model.train(
    data=data_yaml_path,      # Path to dataset YAML
    epochs=30,               # Train for more epochs
    batch=16,                 # Experiment with batch sizes
    imgsz=640,               # Higher resolution images
    device='cuda' if torch.cuda.is_available() else 'cpu',  # Device
    lr0=0.001,               # Initial learning rate
    optimizer='AdamW',       # Optimizer choice (try 'AdamW' for experimentation)
    augment=True,            # Enable augmentations
    freeze=[0, 1, 2],        # Freeze first few layers (adjust based on YOLOv8 architecture)
)

Ultralytics 8.3.94 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/data.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=[0, 1, 2], multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=T

train: Scanning /content/drive/MyDrive/New_shoe/Data-defect/Data-defect/train-def/labels... 1440 images, 720 backgrounds, 4 corrupt: 100%|██████████| 1440/1440 [00:23<00:00, 61.17it/s] 

train: WARNING ⚠️ /content/drive/MyDrive/New_shoe/Data-defect/Data-defect/train-def/images/aug_28_20211201_092710.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
train: WARNING ⚠️ /content/drive/MyDrive/New_shoe/Data-defect/Data-defect/train-def/images/aug_437_20211201_092710.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
train: WARNING ⚠️ /content/drive/MyDrive/New_shoe/Data-defect/Data-defect/train-def/images/aug_732_20211201_092710.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
train: WARNING ⚠️ /content/drive/MyDrive/New_shoe/Data-defect/Data-defect/train-def/images/aug_896_20211201_092710.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0


train: New cache created: /content/drive/MyDrive/New_shoe/Data-defect/Data-defect/train-def/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/drive/MyDrive/New_shoe/Data-defect/Data-defect/val/labels... 180 images, 92 backgrounds, 0 corrupt: 100%|██████████| 180/180 [00:02<00:00, 63.50it/s]


...

                   all        180        116      0.953      0.882      0.948      0.633



30 epochs completed in 1.728 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 22.5MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics 8.3.94 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:15<00:00,  2.57s/it]


                   all        180        116      0.906      0.914      0.938      0.638
Speed: 0.2ms preprocess, 14.4ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/train4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x799a351a1b50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
from ultralytics import YOLO

# Define model path (make sure it's the correct trained model)
model_path = "/content/yolov5/runs/detect/train4/weights/best.pt"

# Load trained model
model = YOLO(model_path)

# Run evaluation on the test dataset
metrics = model.val(data="/content/data.yaml", split="test")

# Print evaluation results (mAP, Precision, Recall, etc.)
print(metrics)


Ultralytics 8.3.94 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /content/drive/MyDrive/New_shoe/Data-defect/Data-defect/test-def/labels... 180 images, 88 backgrounds, 0 corrupt: 100%|██████████| 180/180 [00:02<00:00, 69.76it/s] 


val: New cache created: /content/drive/MyDrive/New_shoe/Data-defect/Data-defect/test-def/labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:32<00:00,  2.69s/it]


                   all        180        131      0.973      0.924      0.975      0.681
Speed: 0.2ms preprocess, 8.4ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/val2
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x799a34ae5390>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.03303

In [ ]:
model_path = '/content/yolov5/runs/detect/train4/weights/best.pt'

# Evaluate the model
model = YOLO(model_path)
results = model.val(data=data_yaml_path)

Ultralytics 8.3.94 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /content/drive/MyDrive/New_shoe/Data-defect/Data-defect/val/labels.cache... 180 images, 92 backgrounds, 0 corrupt: 100%|██████████| 180/180 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:38<00:00,  3.20s/it]


                   all        180        116      0.938      0.908      0.949      0.628
Speed: 0.2ms preprocess, 10.1ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs/detect/val


In [ ]:
import shutil

# Specify the folder you want to zip
folder_path = '/content/yolov5/runs'

# Specify the name of the zip file
zip_filename = 'runs_final.zip'

# Zip the folder
shutil.make_archive(zip_filename, 'zip', folder_path)

# Download the zip file
from google.colab import files
files.download(zip_filename + '.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#  Export model to tflite
from ultralytics import YOLO

# Load a model
model = YOLO('/content/yolov5/runs/detect/train4/weights/best.pt')  # load a custom trained model

# Export the model
model.export(format='tflite')

Ultralytics 8.3.94 🚀 Python-3.11.11 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from '/content/yolov5/runs/detect/train4/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (21.5 MB)
requirements: Ultralytics requirements ['sng4onnx>=1.0.1', 'onnx_graphsurgeon>=0.3.26', 'ai-edge-litert>=1.2.0', 'onnx>=1.12.0', 'onnx2tf>=1.26.3', 'onnxslim>=0.1.31', 'tflite_support', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.7/147.7 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 261.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 138.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 309.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4

100%|██████████| 1.11M/1.11M [00:00<00:00, 27.1MB/s]
Unzipping calibration_image_sample_data_20x128x128x3_float32.npy.zip to /content/yolov5/calibration_image_sample_data_20x128x128x3_float32.npy...: 100%|██████████| 1/1 [00:00<00:00, 44.36file/s]


ONNX: starting export with onnx 1.17.0 opset 19...


ONNX: slimming with onnxslim 0.1.48...
ONNX: export success ✅ 2.3s, saved as '/content/yolov5/runs/detect/train4/weights/best.onnx' (42.8 MB)
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.27.0...
TensorFlow SavedModel: export success ✅ 126.8s, saved as '/content/yolov5/runs/detect/train4/weights/best_saved_model' (107.5 MB)

TensorFlow Lite: starting export with tensorflow 2.18.0...
TensorFlow Lite: export success ✅ 0.0s, saved as '/content/yolov5/runs/detect/train4/weights/best_saved_model/best_float32.tflite' (42.7 MB)

Export complete (130.5s)
Results saved to /content/yolov5/runs/detect/train4/weights
Predict:         yolo predict task=detect model=/content/yolov5/runs/detect/train4/weights/best_saved_model/best_float32.tflite imgsz=640  
Validate:        yolo val task=detect model=/content/yolov5/runs/detect/train4/weights/best_saved_model/best_float32.tflite imgsz=640 data=/content/data.yaml  
Visualize:       https://netron.app


'/content/yolov5/runs/detect/train4/weights/best_saved_model/best_float32.tflite'

In [ ]:
#  Export model to tflite
from ultralytics import YOLO

# Load a model
model = YOLO('/content/yolov5/runs/detect/train4/weights/best.pt')  # load a custom trained model

# Run inference
model.predict('/content/drive/MyDrive/New_shoe/images', save=True, imgsz=640, conf=0.2)


image 1/6 /content/drive/MyDrive/New_shoe/images/aug_469_20211110_185352.jpg: 480x640 1 Defect, 48.8ms
image 2/6 /content/drive/MyDrive/New_shoe/images/aug_471_20211123_192911.jpg: 480x640 (no detections), 13.2ms
image 3/6 /content/drive/MyDrive/New_shoe/images/aug_474_20211110_185612.jpg: 480x640 (no detections), 13.2ms
image 4/6 /content/drive/MyDrive/New_shoe/images/aug_475_20211201_093432.jpg: 480x640 (no detections), 13.2ms
image 5/6 /content/drive/MyDrive/New_shoe/images/aug_476_20211201_094319.jpg: 480x640 (no detections), 13.2ms
image 6/6 /content/drive/MyDrive/New_shoe/images/aug_477_20211201_093612.jpg: 480x640 2 Defects, 13.2ms
Speed: 5.3ms preprocess, 19.1ms inference, 60.1ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'Defect'}
 obb: None
 orig_img: array([[[ 94, 134, 176],
         [100, 140, 182],
         [ 92, 133, 172],
         ...,
         [206, 208, 218],
         [200, 202, 212],
         [208, 210, 220]],
 
        [[ 87, 127, 169],
         [ 92, 132, 174],
         [ 88, 129, 168],
         ...,
         [208, 210, 220],
         [205, 207, 217],
         [213, 215, 225]],
 
        [[ 82, 123, 162],
         [ 85, 126, 165],
         [ 83, 124, 163],
         ...,
         [207, 209, 219],
         [207, 209, 219],
         [215, 217, 227]],
 
        ...,
 
        [[223, 226, 230],
         [220, 223, 227],
         [222, 225, 229],
         ...,
         [210, 215, 218],
         [225, 230, 233],
         [229, 234, 237]],
 
        [[219, 222, 226],
         [220, 223, 227],
         [225, 228, 232],
         ...,
         [209, 214

In [ ]:
# Prediction using custom tflite model

#  Prediction using trained model

from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('/content/yolov5/runs/detect/train4/weights/best_saved_model/best_float32.tflite')
# Run inference
model.predict('/content/drive/MyDrive/New_shoe/images', save=True, imgsz=640, conf=0.2)

Loading /content/yolov5/runs/detect/train4/weights/best_saved_model/best_float32.tflite for TensorFlow Lite inference...

image 1/6 /content/drive/MyDrive/New_shoe/images/aug_469_20211110_185352.jpg: 640x640 1 Defect, 475.4ms
image 2/6 /content/drive/MyDrive/New_shoe/images/aug_471_20211123_192911.jpg: 640x640 (no detections), 459.0ms
image 3/6 /content/drive/MyDrive/New_shoe/images/aug_474_20211110_185612.jpg: 640x640 (no detections), 493.2ms
image 4/6 /content/drive/MyDrive/New_shoe/images/aug_475_20211201_093432.jpg: 640x640 (no detections), 495.0ms
image 5/6 /content/drive/MyDrive/New_shoe/images/aug_476_20211201_094319.jpg: 640x640 (no detections), 443.4ms
image 6/6 /content/drive/MyDrive/New_shoe/images/aug_477_20211201_093612.jpg: 640x640 2 Defects, 471.2ms
Speed: 4.8ms preprocess, 472.9ms inference, 5.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict2


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'Defect'}
 obb: None
 orig_img: array([[[ 94, 134, 176],
         [100, 140, 182],
         [ 92, 133, 172],
         ...,
         [206, 208, 218],
         [200, 202, 212],
         [208, 210, 220]],
 
        [[ 87, 127, 169],
         [ 92, 132, 174],
         [ 88, 129, 168],
         ...,
         [208, 210, 220],
         [205, 207, 217],
         [213, 215, 225]],
 
        [[ 82, 123, 162],
         [ 85, 126, 165],
         [ 83, 124, 163],
         ...,
         [207, 209, 219],
         [207, 209, 219],
         [215, 217, 227]],
 
        ...,
 
        [[223, 226, 230],
         [220, 223, 227],
         [222, 225, 229],
         ...,
         [210, 215, 218],
         [225, 230, 233],
         [229, 234, 237]],
 
        [[219, 222, 226],
         [220, 223, 227],
         [225, 228, 232],
         ...,
         [209, 214